In [14]:
import platform
import warnings
warnings.filterwarnings('ignore')

from os import listdir
from os.path import isfile, join
import math
import itertools
import random
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

random.seed(200)
print('Python version: {}'.format(platform.python_version()))
print('NetworkX version: {}'.format(nx.__version__))
print('Matplotlib version: {}'.format(matplotlib.__version__))
print('Numpy version: {}'.format(np.__version__))

Python version: 3.7.4
NetworkX version: 2.4
Matplotlib version: 3.1.1
Numpy version: 1.18.1


In [3]:
mypath = 'raccoon_proximity_weighted'
files = [mypath + '/' + f for f in listdir(mypath) if isfile(join(mypath, f))]
files = [(int(f.split('_')[5].split('.')[0]), f) for f in files]
files.sort()
#files = [f[1] for f in files]

In [49]:
df = pd.DataFrame()
filt = [str(x) for x in range(16)]

for f in files:
    G = nx.read_graphml(f[1]).subgraph(filt)
    deg = pd.DataFrame({'week':f[0], 'raccoon_ID':list(G), 'degree':list(dict(G.degree).values())})
    DC = nx.degree_centrality(G)
    DC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(DC.keys()),
                           'deg_cent':list(DC.values())})
    BC = nx.betweenness_centrality(G)
    BC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(BC.keys()),
                           'bet_cent':list(BC.values())})
    CC = nx.closeness_centrality(G)
    CC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(CC.keys()),
                           'clos_cent':list(CC.values())})
    EC = nx.eigenvector_centrality(G)
    EC = pd.DataFrame({'week':f[0], 'raccoon_ID':list(EC.keys()),
                           'eig_cent':list(EC.values())})
    
    df_aux = DC.merge(BC, how='outer', on=['week', 'raccoon_ID'])
    df_aux = df_aux.merge(CC, how='outer', on=['week', 'raccoon_ID'])
    df_aux = df_aux.merge(EC, how='outer', on=['week', 'raccoon_ID'])
    df_aux = df_aux.merge(deg, how='outer', on=['week', 'raccoon_ID'])
    df = pd.concat([df,df_aux])
df['raccoon_ID'] = pd.to_numeric(df['raccoon_ID'])
df

,week,raccoon_ID,deg_cent,bet_cent,clos_cent,eig_cent,degree
0,1,1,0.142857,0.000000,0.388889,0.096298,2
1,1,2,0.285714,0.031136,0.451613,0.200650,4
2,1,3,0.428571,0.166667,0.608696,0.361288,6
3,1,4,0.071429,0.000000,0.318182,0.028732,1
4,1,5,0.071429,0.000000,0.400000,0.084649,1
...,...,...,...,...,...,...,...
11,52,11,0.133333,0.016190,0.428571,0.090940,2
12,52,12,0.133333,0.000000,0.384615,0.097801,2
13,52,13,0.133333,0.000000,0.394737,0.127715,2
14,52,14,0.400000,0.135556,0.535714,0.330243,6


In [51]:
mean_by_id = df.drop('week', axis=1).groupby('raccoon_ID').mean().reset_index()
for c in mean_by_id.columns.drop('raccoon_ID'):
    px.bar(mean_by_id, x="raccoon_ID", y=c).show()

In [64]:
mean_by_week = df.drop('raccoon_ID', axis=1).groupby('week').mean().reset_index().melt('week')
px.line(mean_by_week[mean_by_week.variable!='degree'], x='week', y='value', color='variable').show()
px.line(mean_by_week[mean_by_week.variable=='degree'], x='week', y='value', color='variable').show()

In [65]:
max_by_week = df.drop('raccoon_ID', axis=1).groupby('week').max().reset_index().melt('week')
px.line(max_by_week[max_by_week.variable!='degree'], x='week', y='value', color='variable').show()
px.line(max_by_week[max_by_week.variable=='degree'], x='week', y='value', color='variable').show()

In [67]:
min_by_week = df.drop('raccoon_ID', axis=1).groupby('week').min().reset_index().melt('week')
px.line(min_by_week[min_by_week.variable!='degree'], x='week', y='value', color='variable').show()
px.line(min_by_week[min_by_week.variable=='degree'], x='week', y='value', color='variable').show()

In [68]:
random.seed(200)
final_pos = []
for f in files:
    final_pos.append(nx.spring_layout(nx.read_graphml(f)))
final_pos

TypeError: expected str, bytes or os.PathLike object, not tuple

dict_values([2, 4, 6, 1, 1, 4, 3, 5, 8, 8, 6, 1, 3, 3, 1])